In [1]:
import pandas as pd
import random

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [47]:
df = pd.read_parquet("train-00000-of-00002.parquet")

In [48]:
df.head()

,_id,url,title,text
0,20231101.da_508659_0,https://da.wikipedia.org/wiki/Piccadilly%20Cir...,Piccadilly Circus Station,Piccadilly Circus Station er en London Undergr...
1,20231101.da_508659_1,https://da.wikipedia.org/wiki/Piccadilly%20Cir...,Piccadilly Circus Station,Station blev åbnet den 10. marts 1906 af Baker...
2,20231101.da_508659_2,https://da.wikipedia.org/wiki/Piccadilly%20Cir...,Piccadilly Circus Station,"Den gamle stationsbygningen, designet af Lesli..."
3,20231101.da_508659_3,https://da.wikipedia.org/wiki/Piccadilly%20Cir...,Piccadilly Circus Station,Bakerloo line-perronerne på Piccadilly Circus ...
4,20231101.da_508659_4,https://da.wikipedia.org/wiki/Piccadilly%20Cir...,Piccadilly Circus Station,Piccadilly Circus er foreslået som stop på Che...


In [52]:
# For memory management, I will select 100 of those areticles with >9 paragraphs

filtered_articles = df.groupby('title').filter(lambda x: len(x) >= 9)
selected_titles = filtered_articles['title'].drop_duplicates().sample(n=100, random_state=42)
selected_articles = filtered_articles[filtered_articles['title'].isin(selected_titles)]

In [58]:
import random

processed_samples = []

"""The function below will do the following:
Group paragraphs by article title and sort if needed.
Check that each article has at least 9 paragraphs; skip otherwise.
Randomly select a starting index for a 9-paragraph window.
Use the middle (5th) paragraph as the positive sample; surrounding paragraphs are negatives.
Create a sample dictionary, using the article ID and URL from the first row.
Append the processed sample to a list.
"""

for title, group in selected_articles.groupby('title'):
    paragraphs = group['text'].tolist()  
    if len(paragraphs) < 9:
        continue  
    start_idx = random.randint(0, len(paragraphs) - 9)
    selected_window = paragraphs[start_idx:start_idx + 9]
    middle_index = 4  
    positive_sample = selected_window[middle_index]
    negative_samples = selected_window[:middle_index] + selected_window[middle_index + 1:]
    
    sample = {
        'article_id': group['_id'].iloc[0],  
        'title': title,
        'url': group['url'].iloc[0],  
        'positive': positive_sample,
        'negatives': negative_samples
    }

    processed_samples.append(sample)

print("Processing complete. Number of samples:", len(processed_samples))


Processing complete. Number of samples: 100


In [59]:
processed_df = pd.DataFrame(processed_samples)

In [61]:
processed_df.head()

,article_id,title,url,positive,negatives
0,20231101.da_28930_0,2-modulusdeler,https://da.wikipedia.org/wiki/2-modulusdeler,Men har man anvendt en fast fordeler (eng. pre...,[Indenfor den digitale elektronik bruges beteg...
1,20231101.da_720900_0,A.O. Andersen (skibsreder),https://da.wikipedia.org/wiki/A.O.%20Andersen%...,Fra 1919 havde Andersen sæde i Privatbankens b...,[A.O. Andersen fik 1. juli 1890 ansættelse i s...
2,20231101.da_43159_0,ASA Film,https://da.wikipedia.org/wiki/ASA%20Film,A/S Filmatelieret ASA (Oprindelige navn: Aktie...,[ASA-film A/S blev grundlagt i 1936 af Lau Lau...
3,20231101.da_13868_0,Adolf Hitler,https://da.wikipedia.org/wiki/Adolf%20Hitler,Den nationalsocialistiske udenrigspolitik vist...,"[Hitler førte den samme udenrigspolitisk, men ..."
4,20231101.da_996759_0,Amanda Nunes,https://da.wikipedia.org/wiki/Amanda%20Nunes,Nunes var planlagt til at møde Kaitlin Young p...,"[Nunes vandt fem kampe i træk, alle ved en for..."


In [ ]:
processed_df.to_parquet("processed_danish_wikipedia.parquet", index=False)